In [116]:
from pyedflib import highlevel
import os
from os import listdir
from os.path import isfile, join
from pandas import read_csv
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
import numpy as np

In [117]:
eeg_arithmetic_directory = "../datasets/eeg-during-mental-arithmetic-tasks"

In [118]:
# The dataset contains recordings of the students while they were idle and when they were actively solving arithmentic tasks
edf_files_background_eeg = [f for f in listdir(eeg_arithmetic_directory) if isfile(join(eeg_arithmentic_directory, f)) and f.endswith("_1.edf")]
edf_files_recorded_eeg = [f for f in listdir(eeg_arithmetic_directory) if isfile(join(eeg_arithmentic_directory, f)) and f.endswith("_2.edf")]
#data is on format filename: signals, signal_headers, header, remove suffix of the filename
background_eeg = {file[:-6]: highlevel.read_edf(edf_file=join(eeg_arithmetic_directory, file)) for file in edf_files_background_eeg}
recorded_eeg = {file[:-6]: highlevel.read_edf(edf_file=join(eeg_arithmetic_directory, file)) for file in edf_files_recorded_eeg}

In [119]:
# Get labels from subjects
subject_dataframe = read_csv(join(eeg_arithmetic_directory, "subject-info.csv"))
labels= dict(zip(list(subject_dataframe["Subject"]), list(subject_dataframe["Count quality"])))

In [155]:
#(24,21,31000)
# (24, 31000)
#(24, 50)
# reshape to only use measurements from 1 electrode, and 50 samplings
x = [np.array(recorded_eeg[subject][0])[1:2,0:50].reshape(-1) for subject in recorded_eeg.keys()]
# Convert labels to list
y = [labels[subject] for subject in recorded_eeg.keys()]

(36, 50)


In [156]:
# Train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=89)
model = RandomForest()
model.fit(x_train, y_train)

RandomForestClassifier()

In [157]:
y_pred = model.predict(x_test)

In [158]:
print(y_pred)
print(y_test)

[1 0 1 1 1 0 1 0 1 1 1 1]
[0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0]


In [159]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.50      0.67      0.57         3
           1       0.88      0.78      0.82         9

    accuracy                           0.75        12
   macro avg       0.69      0.72      0.70        12
weighted avg       0.78      0.75      0.76        12

